IMPORTS E PARÂMETROS

In [19]:
import tabula as tb
import pandas as pd
import numpy  as np

FUNÇÕES

In [20]:
def normalize_text_java(tables:list) -> list:

    textos_normalizados_java = {
    "1/3 Férias Total Férias": "13_ferias total_ferias", 
    "1/3 Férias"            : "13_ferias", 
    "Total Férias"          : "total_ferias", 
    "13o Salário"           : "13_salario", 
    "TOTAL"                 : "total", 
    "Apropriação"           : "apropriacao", 
    "INSS 20,00%"           : "inss", 
    ".*Ac. Trab. .*"        : "ac_trab", 
    "FGTS 8%"               : "fgts", 
    "SP-PREVCOM"            : "spprevcom", 
    "Total de Encargos"     : "total_encargos", 
    "Férias em pecúnia"     : "ferias_pecunia", 
    "SPPREV"                : "spprev", 
    "Férias"                : "ferias", 
    "\."                    : "",
    ","                     : ".",
    "APROPRIAÇÃO MENSAL"    : "apropriado",
    "REALIZADO MENSAL"      : "realizado",
    "SALDO FINAL MENSAL"    : "saldo",
    "REVERSÃO / REVISÃO DAS APROPRIAÇÕES" : "revertido",
    }
    
    for index, table in enumerate(tables, start=1):
        tables[index-1] = table.replace(textos_normalizados_java, inplace=False, regex=True)
    return tables

def get_table_type(table:list) -> str:
    tipos_tabelas =  [
        "CLT", 
        "AUTÁRQUICO", 
    ]
    
    tipos_tabelas_a_ignorar = [
        "TOTAIS", 
    ]

    if not any(ignore_type in column for column in table for ignore_type in tipos_tabelas_a_ignorar):
        for type in tipos_tabelas:
            if type in table.columns:
                return type
    return None

def split_column(table:list, type:str, col1="Col1", col2="Col2") -> pd.DataFrame:
    df = pd.DataFrame(columns=[col1, col2])
    for value in table[type]:
        values = value.split()
        df.loc[len(df)] = [values[0], values[1] if len(values) > 1 else None]
    return df

def join_columns(table:list, df:pd.DataFrame, type:str) -> pd.DataFrame:
    table.drop(columns=[type], inplace=True)
    table = pd.concat([table, df], axis=1)
    return table

def format_data_frame(df:pd.DataFrame, type:str)->pd.DataFrame:
    colunas_renomeadas = {
        "Unnamed: 0": "tipo", 
        "Unnamed: 1":"Col1", 
        "Col1":"Col2", 
        "Col2":"Col3", 
        "Unnamed: 2":"Col4", 
        "Unnamed: 3":"Col5",
    }

    colunas_ordenadas = [
    "Unnamed: 0", 
    "Unnamed: 1", 
    "Col1", 
    "Col2", 
    "Unnamed: 2", 
    "Unnamed: 3", 
    ]

    df = df.reindex(columns=colunas_ordenadas)
    colunas_renomeadas["Unnamed: 0"] = type
    df = df.rename(columns=colunas_renomeadas)
    return df

def get_line_positions(df:pd.DataFrame, col:str="Col2") -> list:
    tabelas_internas = [
        "apropriado", 
        "realizado", 
        "saldo", 
        "revertido", 
    ]
    
    positions = []
    for index, line in enumerate(df[col]):
        if line in tabelas_internas:
            positions.append(index)
    positions.append(len(df))
    return positions

def split_internal_dataframe(df_main:pd.DataFrame, pos:list):
    results = []
    for i in range(len(pos) - 1):
            df_new  = df_main[pos[i] : pos[i+1]]
            type_df = df_new.iloc[0,2]

            df_new  = df_new.rename(columns=df_new.iloc[1].replace(np.nan, ""))[2:]
            df_new.reset_index(drop=True, inplace=True)
            
            results.append((type_df, df_new))
    return results

def get_dataframes(df_main:pd.DataFrame) -> list:
    type = get_table_type(df_main)
    if type is not None:
        df_main = join_columns(df_main, split_column(df_main, type), type)
        df_main = format_data_frame(df_main, type)
        return split_internal_dataframe(df_main, get_line_positions(df_main))
    else:
        return []

def get_tables_from(file_path):
    output = {}
    tables = normalize_text_java(tb.read_pdf(file_path, pages="all"))
    
    for idx, table in enumerate(tables, start=1):
        results = get_dataframes(table)
        table_results = [] 
        
        for table_type, sub_tables in results:
            table_results.append((table_type, sub_tables))
        
        output[idx] = table_results

    return output

EXECUÇÃO PRINCIPAL

In [21]:
output = get_tables_from("RHPP05LA - GERAL.pdf")

# display(type(output))

for key, value in output.items():
    # display(type(key))
    # display(type(value))
    for tipo_tabela, tabela in value:
        display(tipo_tabela)
        display(tabela)

'apropriado'

,,ferias,13_ferias,total_ferias,13_salario,total
0,apropriacao,60133.58,20044.36,80177.94,60133.58,140311.52
1,inss,12026.48,4008.66,16035.14,9991.98,26027.12
2,ac_trab,107470.44,35822.98,143293.42,89290.32,232583.74
3,fgts,4810.46,1603.31,6413.77,3996.63,10410.40
4,spprevcom,0.00,0.00,0.00,718.24,718.24
5,total_encargos,124307.38,41434.95,165742.33,103997.17,269739.50
6,total,184440.96,61479.31,245920.27,164130.75,410051.02


'realizado'

,,ferias,13_ferias,total_ferias,13_salario,total
0,apropriacao,80594.28,28188.65,108782.93,0.00,108782.93
1,ferias_pecunia,15287.07,0.00,15287.07,0.00,15287.07
2,Total,95881.35,28188.65,124070.00,0.00,124070.00
3,inss,19176.23,5637.66,24813.89,0.00,24813.89
4,ac_trab,171359.06,50378.68,221737.74,0.00,221737.74
5,fgts,7670.44,2255.00,9925.44,0.00,9925.44
6,spprevcom,0.00,0.00,0.00,0.00,0.00
7,total_encargos,198205.73,58271.34,256477.07,0.00,256477.07
8,total,294087.08,86459.99,380547.07,0.00,380547.07


'saldo'

...

,,ferias,13_ferias,total_ferias,13_salario,total
0,apropriacao,146256.84,49705.68,195962.52,0.00,195962.52
1,spprevcom,0.00,0.00,0.00,0.00,0.00
2,total,146256.84,49705.68,195962.52,0.00,195962.52
3,spprev,0.00,0.00,0.00,0.00,0.00


'saldo'

,,ferias,13_ferias,total_ferias,13_salario,total
0,apropriacao,143681.76,47781.35,191463.11,261645.95,453109.06
1,spprevcom,0.00,0.00,0.00,164.54,164.54
2,total,143681.76,47781.35,191463.11,261810.49,453273.60
3,spprev,0.00,0.00,0.00,376284.78,376284.78


'revertido'

,,ferias,13_ferias,total_ferias,13_salario,total
0,apropriacao,93989.58,31441.96,125431.54,0.00,125431.54
1,spprevcom,0.00,0.00,0.00,0.00,0.00
2,total,93989.58,31441.96,125431.54,0.00,125431.54
3,spprev,0.00,0.00,0.00,0.00,0.00
